In [7]:
import fcntl
import os
import platform
from tkinter import Tk
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import scipy.stats as stats
from sklearn.cluster import KMeans

import sys
sys.path.insert(0, os.path.abspath('.'))
from tools import Sim_Parameters, Train_Parameters, create_dataset, RMSELoss, train_val_test, test_epoch, simulator, ballsINTObins, load_data, NoisyDataset


In [8]:


def solve_for_B(A, C):
    # Calculate the Moore-Penrose pseudoinverse of matrix A
    A_pseudo_inv = np.linalg.pinv(A)
    
    # Calculate matrix B
    B = np.dot(A_pseudo_inv, C)
    
    return B

# # Example matrices A and C (you can replace these with your own matrices)
# A = np.array([[1, 2], [3, 4], [5, 6]])
# C = np.array([[7, 8], [9, 10], [11, 12]])

# # Solve for matrix B
# B = solve_for_B(A, C)

# print("Matrix B:")
# print(B)



substance_ind_list = [0, 1, 2, 3]
substance_ind_list.sort()

basis_func_ind_list = [0, 1, 3, 6]
# basis_func_ind_list = [0, 1, 2, 3, 4]
basis_func_ind_list.sort()

temp_K_list = np.asarray([293.15])
temp_K_list.sort()


air_trans, basis_funcs, spectra, substances_emit = load_data(air_trans_file='./data/Test 3 - 4 White Powers/Air transmittance.xlsx',
                                                             basis_func_file='./data/Test 3 - 4 White Powers/Basis functions_4-20um.xlsx',
                                                             spectra_file='./data/Test 3 - 4 White Powers/white_powders_spectra.xlsx', 
                                                             substances_emit_file='./data/Test 3 - 4 White Powers/white_powders.xlsx',)


substance_names = np.array(pd.read_excel('./data/Test 3 - 4 White Powers/white_powders_names.xlsx', header=None))

b_f_comb = [0, 1, 3, 6]
comb = b_f_comb
temp_K = 293.15

sim_params = Sim_Parameters(air_trans=air_trans,
                            air_RI=1,
                            atm_dist_ratio=0.11,
                            basis_funcs=basis_funcs,
                            basis_func_comb=comb,
                            substance_ind_list=substance_ind_list,
                            spectra=spectra,
                            substances_emit=substances_emit,
                            temp_K=temp_K)

dataset = create_dataset(sim_params)

In [9]:
sensor_output_list = []

for idx in substance_ind_list:
    sub_signal = substances_emit[:, idx]
    sub_signal = np.expand_dims(sub_signal, 1)
    
    out = simulator(sim_params, sub_signal)  # Apply simulator() function to the column
    sensor_output_list.append(out)

sensor_output_list = np.asarray(sensor_output_list).transpose()
print(sensor_output_list.shape)


(4, 4)


In [10]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

torch.manual_seed(28)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# test_noise_perc_list = [0, 0.01, 0.03, 0.05, 0.1, 0.2, 0.4, 1, 2]
test_noise_perc_list = [i * 0.05 for i in range(41)]

In [11]:
results = []

for test_noise_perc in test_noise_perc_list:
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    if test_noise_perc != 0:
        test_dataset = NoisyDataset(test_dataset, test_noise_perc)


    dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    loss_list = []
    targ_list = []
    pred_list = []

    

    for batch_idx, (x, y) in enumerate(dataloader):
        print(batch_idx)
        print('x - C - mixture - Norm', x)
        print('y - B - ratio', y)

        print('A - shape', sensor_output_list.shape)
        print('B - shape', y.numpy().transpose().shape)
        print('C - shape', x.numpy().transpose().shape)

        X = np.matmul(sensor_output_list, y.numpy().transpose())
        print('Cal C\n', X.transpose())
        print('Cal C - shape', X.shape)
        print('C original', x.numpy())

        Cal_B_by_Cal_C = solve_for_B(sensor_output_list, X)
        print('Cal B by Cal C\n', Cal_B_by_Cal_C)

        prediction = solve_for_B(sensor_output_list, x.numpy().transpose())
        print('pred', prediction.transpose())
        print('sum of pred', np.sum(prediction))
        normalized_prediction = prediction / np.sum(prediction)
        prediction = normalized_prediction
        print('pred', prediction.transpose())
        print('sum of pred', np.sum(prediction))

        criterion = nn.L1Loss()

        loss = criterion(torch.from_numpy(prediction.transpose()), y)
        print('loss', loss)
        loss_list.append(loss.numpy())
        targ_list.append(np.squeeze(y.numpy()))
        pred_list.append(np.squeeze(prediction))

    avg_loss = np.mean(loss_list)
    targ_list = np.asarray(targ_list)
    pred_list = np.asarray(pred_list)
    # print('avg loss', avg_loss)
    # print(pred_list.shape)
    # print(targ_list.shape)
    diff_list = pred_list - targ_list

    print('diff_list', diff_list)


    row = {
        'Temperature_K': temp_K,
        'Substance Number': len(substance_ind_list),
        'Substance Comb': tuple(substance_ind_list),
        'Basis Function Number': len(comb),
        'Comb of Basis Functions': comb,
        'Train Noise Max Percentage': 0,
        'Test Noise Max Percentage': test_noise_perc,
        'L1Loss': avg_loss
    }



    perc95_interval = []
    for i, sub_ind in enumerate(substance_ind_list):

        data = diff_list[:, i]
        # params = stats.norm.fit(data)
        # data_mean, data_std = params
        ae, loce, scalee = stats.skewnorm.fit(data)
        
        # Add mean and std to row
        # row[f'Substance {sub_ind} skewnorm a'] = ae
        # row[f'Substance {sub_ind} skewnorm loc'] = loce
        # row[f'Substance {sub_ind} skewnorm scale'] = scalee


        # Create a skewed normal distribution object
        dist = stats.skewnorm(ae, loce, scalee)


        for confidence_perc in (0.68, 0.95, 0.997):
            # Calculate the quantiles for a specific percentage
            lower_quantile = dist.ppf(0.5-confidence_perc/2)  # Lower quantile covering % of the data
            upper_quantile = dist.ppf(0.5+confidence_perc/2)  # Upper quantile covering % of the data

            # row[f'Substance {sub_ind} skewnorm {confidence_perc*100}% low'] = lower_quantile
            # row[f'Substance {sub_ind} skewnorm {confidence_perc*100}% high'] = upper_quantile
            # row[f'Substance {sub_ind} skewnorm {confidence_perc*100}% interval range'] = upper_quantile - lower_quantile

            if confidence_perc == 0.95:
                perc95_interval.append(upper_quantile - lower_quantile)


    # perc95_min = np.min(perc95_interval)
    # row[f'MIN skewnorm 95% interval range'] = perc95_min
    # perc95_max = np.max(perc95_interval)
    # row[f'MAX skewnorm 95% interval range'] = perc95_max
    perc95_avg = np.mean(perc95_interval)
    row[f'AVG skewnorm 95% interval range'] = perc95_avg


    # Append the row to the results list
    results.append(row)
# Create a pandas DataFrame from the results list
df = pd.DataFrame(results)



# df.to_csv('.csv', index=False)

output_file_name = 'loss_test_mtx_m.csv'
# Check if the file "loss.pkl" already exists
if os.path.isfile(output_file_name):

    # If the file exists, load the existing DataFrame from the file
    df_existing = pd.read_csv(output_file_name)

    # Open the file in binary mode
    with open(output_file_name, 'w') as f:
        while True:
            try:
                # Get a lock on the file
                fcntl.flock(f, fcntl.LOCK_EX | fcntl.LOCK_NB)
                break
            except IOError:
                # Lock is held by another process, wait and try again
                time.sleep(1)

        # Merge df to df_existing, and if there's duplicates, keep the new values from df
        df = pd.concat([df, df_existing]).drop_duplicates(
            subset=['Temperature_K', 'Substance Number', 'Substance Comb', 'Basis Function Number', 
                    'Comb of Basis Functions', 'Train Noise Max Percentage', 'Test Noise Max Percentage'], keep='first')
        df = df.sort_values(by=list(df.columns))
        print(df)

        # Save the DataFrame to a CSV file "loss.csv"
        df.to_csv(output_file_name, index=False)

        # Release the lock on the file
        fcntl.flock(f, fcntl.LOCK_UN)
else:
    # Save the DataFrame to a CSV file "loss.csv"
    df.to_csv(output_file_name, index=False) 



0
x - C - mixture - Norm tensor([[14.2358, 15.1640, 13.4257, 12.2038]])
y - B - ratio tensor([[0.1000, 0.0000, 0.3000, 0.6000]])
A - shape (4, 4)
B - shape (4, 1)
C - shape (4, 1)
Cal C
 [[14.23580247 15.16403458 13.42571822 12.20382657]]
Cal C - shape (4, 1)
C original [[14.235802 15.164034 13.425717 12.203826]]
Cal B by Cal C
 [[1.00000001e-01]
 [1.42108547e-14]
 [3.00000012e-01]
 [6.00000024e-01]]
pred [[ 1.00000639e-01 -4.48832942e-07  2.99998821e-01  6.00000570e-01]]
sum of pred 0.9999995806953308
pred [[ 1.00000681e-01 -4.48833131e-07  2.99998947e-01  6.00000821e-01]]
sum of pred 1.0
loss tensor(7.4750e-07, dtype=torch.float64)
1
x - C - mixture - Norm tensor([[12.8861, 13.4344, 11.5024, 11.7016]])
y - B - ratio tensor([[0.0000, 0.6000, 0.1000, 0.3000]])
A - shape (4, 4)
B - shape (4, 1)
C - shape (4, 1)
Cal C
 [[12.88609012 13.43440055 11.50235737 11.70161979]]
Cal C - shape (4, 1)
C original [[12.886089 13.4344   11.502357 11.701619]]
Cal B by Cal C
 [[-1.42108547e-14]
 [ 6.000